<a href="https://colab.research.google.com/github/gchaewon/Euron5th_BusTayo/blob/main/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울시 버스 승하차 예측 모델링

서울시 버스 승하차 예측 모델링

- LightGBM 모델을 사용

- 서울시 버스 승하차 인원 데이터 중 train – 22/9 ~ 23/8, test – 23/9 ~ 23/11 기간으로 하여 전처리한 데이터로 모델 학습

- scaler에 따른 정확도 비교도 진행


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')

## 0. 데이터 로딩

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = '/content/drive/MyDrive/Euron/프로젝트/'

train = pd.read_csv(path+'train.csv')

print(train.shape)
train.head(3)

(450530, 23)


,month,bus_id,station_name,type,bus_station_ID,6~8_ride,6~8_takeoff,8~10_ride,8~10_takeoff,10~12_ride,...,14~16_ride,14~16_takeoff,16~18_ride,16~18_takeoff,18~20_ride,18~20_takeoff,20~22_ride,20~22_takeoff,rain,temp
0,202209,741,종로2가사거리(00077),서울간선버스,77,654,354,894,559,809,...,576,769,563,781,574,800,576,776,201.5,22.4
1,202209,470,종로2가사거리(00067),서울간선버스,67,788,325,1232,626,1265,...,1217,696,1157,815,1074,982,982,954,201.5,22.4
2,202209,100,창경궁.서울대학교병원(00031),서울간선버스,31,227,708,566,1139,677,...,282,504,208,449,193,450,272,458,201.5,22.4


In [6]:
test = pd.read_csv(path+'test.csv')

print(test.shape)
test.head(3)

(125576, 23)


,month,bus_id,station_name,type,bus_station_ID,6~8_ride,6~8_takeoff,8~10_ride,8~10_takeoff,10~12_ride,...,14~16_ride,14~16_takeoff,16~18_ride,16~18_takeoff,18~20_ride,18~20_takeoff,20~22_ride,20~22_takeoff,rain,temp
0,202309,N37,종로2가사거리(00032),서울심야버스,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,134.5,23.7
1,202309,470,종로2가사거리(00066),서울간선버스,66,699,349,1160,629,1292,...,1191,788,1130,904,988,882,886,883,134.5,23.7
2,202309,N37,종로2가사거리(00089),서울심야버스,89,0,0,0,0,0,...,0,0,0,0,0,0,0,0,134.5,23.7


## 1. lgbm모델 학습

In [18]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [7]:
# 배치 처리를 위한 함수 정의
def batch_generator(X, y, batch_size=1000):
    num_samples = X.shape[0]
    num_batches = num_samples // batch_size + 1

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_samples)
        yield X.iloc[start_idx:end_idx], y.iloc[start_idx:end_idx]

In [11]:
# 스케일링 함수 정의
def scaler(type, train, test):
    if type == 'standard':
        scaler = StandardScaler()
    elif type == 'minmax':
        scaler = MinMaxScaler()
    elif type == 'robust':
        scaler = RobustScaler()

    train_scaled = pd.DataFrame(data=scaler.fit_transform(train), columns=train.columns)
    test_scaled = pd.DataFrame(data=scaler.fit_transform(test), columns=test.columns)

    train_scaled = train_scaled.astype('float')
    test_scaled = test_scaled.astype('float')
    X_train = train_scaled.drop(columns=['18~20_ride'])
    y_train = train_scaled['18~20_ride']

    X_test = test_scaled.drop(columns=['18~20_ride'])
    y_test = test_scaled['18~20_ride']

    return X_train, X_test, y_train, y_test

In [15]:
# 모델 학습 함수 정의
def modeling(X_train, X_test, y_train, y_test, type):
    lgbm = lgb.LGBMRegressor(num_iterations = 1000,
                                learning_rate = 0.05,
                                boosting = 'dart',
                         Metric = 'regression_l2', n_jobs=-1)
    # tqdm을 사용하여 학습 진행률 표시
    for i, (X_batch, y_batch) in enumerate(tqdm(batch_generator(X_train, y_train), total=len(X_train) // 1000 + 1)):
        lgbm.fit(X_batch, y_batch)

        # 각 회차의 MSE 계산
        if i % 10 == 0:  # 매 10회차마다 MSE 출력
            y_pred = lgbm.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            print(f"Iteration {i}: Test MSE = {mse:.4f}")

    # 테스트 데이터로 예측 수행
    y_pred = lgbm.predict(X_test)

    # 최종 MSE 출력
    mse = mean_squared_error(y_test, y_pred)
    print(f'{type} scaling MSE: {mse:.4f}')

In [16]:
# 카테고리 데이터를 숫자형으로 변환
train['bus_id'] = pd.factorize(train['bus_id'])[0]
train['station_name'] = pd.factorize(train['station_name'])[0]
train['type'] = pd.factorize(train['type'])[0]

test['bus_id'] = pd.factorize(test['bus_id'])[0]
test['station_name'] = pd.factorize(test['station_name'])[0]
test['type'] = pd.factorize(test['type'])[0]

# 피처와 타겟 값 분리
X_train = train.drop(columns=['18~20_ride'])
y_train = train['18~20_ride']

X_test = test.drop(columns=['18~20_ride'])
y_test = test['18~20_ride']

### 스케일링 없이 모델 학습

without scaling MSE: 74410.8469

In [19]:
modeling(X_train, X_test, y_train, y_test, 'without')

  0%|          | 0/451 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


  0%|          | 1/451 [00:17<2:14:47, 17.97s/it]

Iteration 0: Test MSE = 21576.6334
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


  2%|▏         | 11/451 [01:17<57:08,  7.79s/it]

...

 98%|█████████▊| 441/451 [45:03<01:18,  7.84s/it]

Iteration 440: Test MSE = 12477.2619
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


100%|██████████| 451/451 [45:57<00:00,  6.11s/it]

Iteration 450: Test MSE = 74410.8469


without scaling MSE: 74410.8469


## 2. 스케일링 후 lgbm 모델 학습

In [20]:
# 카테고리 데이터를 숫자형으로 변환
train['bus_id'] = pd.factorize(train['bus_id'])[0]
train['station_name'] = pd.factorize(train['station_name'])[0]
train['type'] = pd.factorize(train['type'])[0]

test['bus_id'] = pd.factorize(test['bus_id'])[0]
test['station_name'] = pd.factorize(test['station_name'])[0]
test['type'] = pd.factorize(test['type'])[0]

### 1) standard scaler

standard scaling MSE: 0.3628

In [ ]:
X_train, X_test, y_train, y_test = scaler('standard', train, test)
modeling(X_train, X_test, y_train, y_test, 'standard')

  0%|          | 0/451 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000275 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4272
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 19
[LightGBM] [Info] Start training from score 0.426077
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


  0%|          | 1/451 [00:08<1:04:47,  8.64s/it]

Iteration 0: Test MSE = 0.1095
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000212 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4277
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 19
[LightGBM] [Info] Start training from score 0.286215


  0%|          | 2/451 [00:11<37:50,  5.06s/it]  

...

100%|█████████▉| 450/451 [25:23<00:02,  2.77s/it]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1732
[LightGBM] [Info] Number of data points in the train set: 530, number of used features: 19
[LightGBM] [Info] Start training from score -0.586320
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 451/451 [25:29<00:00,  3.39s/it]

Iteration 450: Test MSE = 0.3628
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


standard scaling MSE: 0.3628


### 2) Min-Max scaler

minmax scaling MSE: 0.0002

In [ ]:
X_train, X_test, y_train, y_test = scaler('minmax', train, test)
modeling(X_train, X_test, y_train, y_test, 'minmax')

  0%|          | 0/451 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4270
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 19
[LightGBM] [Info] Start training from score 0.027918
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


  0%|          | 1/451 [00:12<1:34:19, 12.58s/it]

Iteration 0: Test MSE = 0.0001
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.065961 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4274
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 19
[LightGBM] [Info] Start training from score 0.024695


  0%|          | 2/451 [00:16<56:33,  7.56s/it]  

...

100%|█████████▉| 450/451 [41:38<00:04,  4.97s/it]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050782 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1708
[LightGBM] [Info] Number of data points in the train set: 530, number of used features: 19
[LightGBM] [Info] Start training from score 0.004589
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

100%|██████████| 451/451 [41:47<00:00,  5.56s/it]

Iteration 450: Test MSE = 0.0002
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


minmax scaling MSE: 0.0002


### 3) Robust scaler

robust scaling MSE: 0.4487

In [ ]:
X_train, X_test, y_train, y_test = scaler('robust', train, test)
modeling(X_train, X_test, y_train, y_test, 'robust')

  0%|          | 0/451 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000169 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4270
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 19
[LightGBM] [Info] Start training from score 0.808424
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


  0%|          | 1/451 [00:08<1:06:56,  8.93s/it]

Iteration 0: Test MSE = 0.1341
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000177 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4275
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 19
[LightGBM] [Info] Start training from score 0.651672


  0%|          | 2/451 [00:11<38:32,  5.15s/it]  

...

100%|█████████▉| 450/451 [25:12<00:02,  2.69s/it]

[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000124 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1729
[LightGBM] [Info] Number of data points in the train set: 530, number of used features: 19
[LightGBM] [Info] Start training from score -0.326229
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

100%|██████████| 451/451 [25:18<00:00,  3.37s/it]

Iteration 450: Test MSE = 0.4487
[LightGBM] [Warning] Unknown parameter: Metric
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


robust scaling MSE: 0.4487


In [ ]:
X_train, X_test, y_train, y_test = scaler('robust', train, test)
modeling(X_train, X_test, y_train, y_test, 'robust')

## 3. linear regression 모델 학습

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 선형 회귀 모델 생성
model = LinearRegression()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터로 예측 수행
y_pred = model.predict(X_test)

# 평가 지표 계산 (평균 제곱 오차)
mse = mean_squared_error(y_test, y_pred)
print(f'linear regression scaling MSE: {mse:.4f}')

linear regression scaling MSE: 0.0059
